In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [ ]:
df = pd.read_pickle('country_data.pkl')
all_countries = pd.DataFrame()

for c,v in df.items():
    temp = pd.DataFrame({c:v.iloc[-1,:]}).T
    all_countries = all_countries.append(temp)

all_countries.loc['UK',:] = all_countries.loc['United Kingdom',:]
all_countries.drop(['conf_over_dead','dead_conf_ratio'],axis=1,inplace=True)
mask = all_countries['deceased'] > 0
all_countries = all_countries[mask]

all_countries['conf_log'] = np.log10(all_countries['confirmed'])
all_countries['dead_log'] = np.log10(all_countries['deceased'])

all_countries

In [ ]:
world_facts = pd.read_csv('world_density.csv',sep=';',usecols=[1,2,4,5],index_col=0)
world_facts['pop2020'] *= 1000 # uses dot a thousand separator
world_facts

In [ ]:
def strip(x):
    return x.replace('$','').replace(',','')

world_gdp = pd.read_csv('world_gdp.csv',sep=';',header=None,usecols=[1,6],
                        converters={'GDPpc': strip},names=['state','GDPpc'],index_col='state')

world_gdp['GDPpc'] = world_gdp['GDPpc'].astype(int)
world_facts = world_facts.join(world_gdp)
world_facts.rename(columns={'GDPpc' : 'gdp'},inplace=True)
world_facts.loc['US',:] = world_facts.loc['United States',:]
world_facts.loc['UK',:] = world_facts.loc['United Kingdom',:]
world_facts.loc['UK']

In [ ]:
world_age = pd.read_csv('world_median_age.csv',sep=r'\t',header=None,
                        usecols=[1,2],index_col=0,names=['state','median_age'],engine='python')

world_facts = world_facts.join(world_age)
world_facts.loc['US']

In [ ]:
abb = pd.read_csv('country_abb.csv',sep=';',header=None,index_col=1,names=['abb','state'])
world_facts = world_facts.join(abb)

all_countries = all_countries.join(world_facts)
all_countries['density_log'] = np.log10(all_countries['density'])
all_countries['gdp_log'] = np.log10(all_countries['gdp'])

all_countries.dropna(inplace=True)
all_countries.sort_values('median_age',inplace=True)
all_countries.sort_values('median_age')

In [ ]:
all_countries.describe()

In [ ]:
import scipy.stats as sps

all_countries.sort_values('gdp',ascending=False,inplace=True)

slope,intercept,_,_,_ = sps.linregress(range(len(all_countries['gdp'])),all_countries['gdp'])

In [ ]:
quartiles = np.percentile(all_countries['gdp'],[25,50,75])
print (quartiles)

plt.figure(figsize=(18,12))
plt.bar(all_countries.index,(all_countries['gdp']))
plt.xticks(rotation=90)

X = range(1,len(all_countries) +1 )

plt.plot(X,np.power(10,np.log10(X * slope + intercept)), color='r',ls='dashed')
plt.yscale('log')

max_gdp = all_countries['gdp'].argmax()
print (all_countries.iloc[max_gdp])

first_q = all_countries['gdp'].sort_values().where(all_countries['gdp'] < quartiles[0]).dropna()
second_q = all_countries['gdp'].sort_values().where(all_countries['gdp'] > first_q[first_q.argmax()]).dropna()

print (first_q)
print (second_q)

In [ ]:
cats = 4

gdp_cat = pd.qcut(all_countries['gdp'],q=cats,labels=range(1,cats + 1))
all_countries['gdp_cat'] = gdp_cat

#all_countries = (all_countries.loc[all_countries['gdp_cat'] >= 9]).copy()
all_countries

In [ ]:
all_countries.describe()

In [ ]:
all_countries.to_pickle('world_facts.pkl')


In [ ]:
plt.figure(figsize=(18,12))
sns.violinplot(data=all_countries,inner='box',x='gdp_cat',y='gdp',scale='count')